In [1]:
from time import sleep
from random import randint

In [2]:
artist = 'Weezer'
song   = 'Island in the Sun'

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [4]:
def find_link_from_search(artist, song):
    # url = 'https://www.ultimate-guitar.com/search.php?search_type=title&value=weezer%20island%20in%20the%20sun'
    # search UG for artist and song
    
    if ' - ' in song:
        song = song.split(' - ')[0]
    
    url = 'https://www.ultimate-guitar.com/search.php?search_type=title&value='+artist+'%20'+song

    # get the soup
    sleep(randint(1,4))
    text = requests.get(url).text
    soup = BeautifulSoup(text)

    # do some crafty splitting to get the tab URL
    # first few results are often ads...
    searchResults = str(soup).split('"tab_url":"')[3:6] 

    new_link = str(searchResults).split("},{")[0][2:-1]

    return new_link

In [5]:
def parse_title(url):
    artist_and_title = url.split('tab/')[1]  

    if '/' in artist_and_title:
        artist = artist_and_title.split('/')[0]  
        title  = artist_and_title.split('/')[1]
        title  = title.split('-ch')[0]
    else:
        artist = 'unknown'
        title = 'unknown'
    
    return artist, title

In [6]:
def scrape(url): 
    output_list = []
    chordlist   = []
    
    sleep(randint(1,4))
    response = requests.get(url)
    html_soup = BeautifulSoup(response.text, 'lxml')

    lessSoupy = html_soup.find('div', class_='js-store')
    stringySoup = str(lessSoupy)

    splitSoup = stringySoup.split('[ch]')[1:-1]

    for ielement in range(0, len(splitSoup)):
        chordlist.append(str(splitSoup[ielement]).split('[/ch')[0])
        
    artist_name, song_name = parse_title(url)
    output_list.append(("Song", song_name))
    output_list.append(("Artist", artist_name))

#     allChords = ("All Chords", chords)
    #output_list.append(allChords)

    chords = set(chordlist)
    chords = ",".join(chords)
    chords = ("Chords", chords)
    output_list.append(chords)
#     output_list.append(allChords)

    song_info = dict(output_list)

    return song_info

In [7]:
new_link = find_link_from_search(artist, song)
print(new_link)

song_info = scrape(new_link)
dfSong = pd.DataFrame.from_dict(song_info, orient='index').transpose()

dfSong

https://tabs.ultimate-guitar.com/tab/weezer/island-in-the-sun-chords-852868


,Song,Artist,Chords
0,island-in-the-sun,weezer,"Am,Em,D,C,G"


In [8]:
# get spotify credentials
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_credentials_manager = SpotifyClientCredentials(client_id= '6426e18767ac42d78a9c08f41cacba65',
                                                     client_secret= '4a2cdab50e1c4ecd9fc62cae20664e29')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
pl_link = 'https://open.spotify.com/playlist/37i9dQZF1DWXRqgorJj26U?si=sYkoH3ZXSlKrMFtmVSp85Q'
pl_id   = pl_link.split('playlist/')[1]
pl_id   = pl_id.split('?')[0]

In [10]:
# response = sp.playlist_items(pl_id,
#                              offset=offset,
#                              fields='items.track.id,total',
#                              additional_types=['track'])

# results = spotify.user_playlist(username, playlist_id,
#                                 fields='tracks,next,name')
results = sp.user_playlist('Spotify', pl_id,
                                fields='tracks,next,name')

pl_tracks  = []
pl_artists = []

for nItem in range(0, len(results['tracks']['items'])):
    pl_tracks.append(results['tracks']['items'][nItem]['track']['name'])
    pl_artists.append(results['tracks']['items'][nItem]['track']['artists'][0]['name'])

In [12]:
pl_tab_links = []

for iTrack in range(0, len(pl_tracks)):
    tmp_link = find_link_from_search(pl_artists[iTrack], pl_tracks[iTrack])
    pl_tab_links.append(tmp_link)

In [13]:
while("" in pl_tab_links) : 
    pl_tab_links.remove("") 
    
# pl_tab_links

['https://tabs.ultimate-guitar.com/tab/jimi-hendrix/all-along-the-watchtower-chords-810679',
 'https://tabs.ultimate-guitar.com/tab/acdc/back-in-black-chords-18953',
 'https://tabs.ultimate-guitar.com/tab/david-bowie/rebel-rebel-chords-1397520',
 'https://tabs.ultimate-guitar.com/tab/the-clash/should-i-stay-or-should-i-go-chords-27335',
 'https://tabs.ultimate-guitar.com/tab/aerosmith/sweet-emotion-chords-1787420',
 'https://tabs.ultimate-guitar.com/tab/queen/another-one-bites-the-dust-chords-1090617',
 'https://tabs.ultimate-guitar.com/tab/the-doors/light-my-fire-chords-15032',
 'https://tabs.ultimate-guitar.com/tab/zz-top/la-grange-chords-2307723',
 'https://tabs.ultimate-guitar.com/tab/bob-dylan/like-a-rolling-stone-chords-138850',
 'https://tabs.ultimate-guitar.com/tab/cream/sunshine-of-your-love-chords-64026',
 'https://tabs.ultimate-guitar.com/tab/creedence-clearwater-revival/fortunate-son-chords-15889',
 'https://tabs.ultimate-guitar.com/tab/janis-joplin/me-and-bobby-mcgee-chord

In [21]:
# del pl_tab_links[15]
pl_tab_links

['https://tabs.ultimate-guitar.com/tab/jimi-hendrix/all-along-the-watchtower-chords-810679',
 'https://tabs.ultimate-guitar.com/tab/acdc/back-in-black-chords-18953',
 'https://tabs.ultimate-guitar.com/tab/david-bowie/rebel-rebel-chords-1397520',
 'https://tabs.ultimate-guitar.com/tab/the-clash/should-i-stay-or-should-i-go-chords-27335',
 'https://tabs.ultimate-guitar.com/tab/aerosmith/sweet-emotion-chords-1787420',
 'https://tabs.ultimate-guitar.com/tab/queen/another-one-bites-the-dust-chords-1090617',
 'https://tabs.ultimate-guitar.com/tab/the-doors/light-my-fire-chords-15032',
 'https://tabs.ultimate-guitar.com/tab/zz-top/la-grange-chords-2307723',
 'https://tabs.ultimate-guitar.com/tab/bob-dylan/like-a-rolling-stone-chords-138850',
 'https://tabs.ultimate-guitar.com/tab/cream/sunshine-of-your-love-chords-64026',
 'https://tabs.ultimate-guitar.com/tab/creedence-clearwater-revival/fortunate-son-chords-15889',
 'https://tabs.ultimate-guitar.com/tab/janis-joplin/me-and-bobby-mcgee-chord

In [22]:
# df_pl = pd.DataFrame()
df_pl = []

for iLink in range(0, len(pl_tab_links)):
    song_info = scrape(pl_tab_links[iLink])
    dfSong = pd.concat([dfSong, pd.DataFrame.from_dict(song_info, orient='index').transpose()], axis=0)

dfSong = dfSong.reset_index(drop=True)
dfSong

,Song,Artist,Chords
0,island-in-the-sun,weezer,"Am,Em,D,C,G"
1,all-along-the-watchtower,jimi-hendrix,"Bb,Cm,Ab"
2,back-in-black,acdc,"A,D,C,B,E,G,A/C#"
3,rebel-rebel,david-bowie,"E,A,Bm,D"
4,should-i-stay-or-should-i-go,the-clash,"A,G,D,F"
5,sweet-emotion,aerosmith,"A,C,G,D"
6,another-one-bites-the-dust,queen,"Am,Em,A,C,B,G,F#m"
7,light-my-fire,the-doors,"Am7,D#,A#,A,D,Bm7,B,E,G#,C,F,G,F#m"
8,la-grange,zz-top,"A,D,C,F,Eb"
9,like-a-rolling-stone,bob-dylan,"Em,C,Dm7,F,Dm,G"


In [25]:
# # scrape links one by one for chords, save out a dictionary when successful
# for iSong in range(0, len(link_df)):
#     url = link_df['song_links'][iSong]
#     song_info = scrape(url, skill[iLevel])

#     if song_info['Chords']:
#         # set up name string
#         tmpName = [song_info['Artist'] + ' - ' + song_info['Song']]
#         fname = ' ' 
#         fname = fname.join(tmpName)

#         # open, write, save a csv under "Aritst - Song"
# #                 with open('ug_chords/'+fname+'.csv', 'w') as f:
#         with open('ug_chords10/'+fname+'.csv', 'w') as f:
#             for key in song_info.keys():
#                 f.write("%s,%s\n"%(key,song_info[key]))
                

dfSong.to_csv('playlist_df.csv', index=False)

In [27]:
# put together a list of various standard major & minor chords
sl1 = ['A', 'Bb', 'B', 'C', 'C#', 'Db', 'D', 'Eb', 'E', 'F', 'F#', 'G', 'G#', 'Ab' ]
sl2 = [s + 'maj' for s in sl1]
sl3 = [s + 'm' for s in sl1]
sl4 = [s + 'min' for s in sl1]

standardList = sl1+sl2+sl3+sl4

In [28]:
# get feature dataframe for the known song.
nUniqueChords   = []
nHardChords     = []
difficultyLabel = []

spotFeatures = {}

# spotFeatures['key'] = []
spotFeatures['tempo'] = []
# spotFeatures['duration_ms'] = []
spotFeatures['time_signature'] = []
spotFeatures['energy'] = []

for iSong in range(0, len(dfSong)):
    songName   = dfSong['Song'][iSong].replace('-', ' ').title()
    artistName = dfSong['Artist'][iSong].replace('-', ' ').title()

    # search spotify api
    result = sp.search(songName, limit=10) 
    nItems = len(result['tracks']['items'])

    # match artist info, and get URI
    for iArtist in range(0, nItems):    
        if result['tracks']['items'][iArtist]['artists'][0]['name'] == artistName:
           songURI = result['tracks']['items'][iArtist]['uri']

    # if we successfully found the song in spotify, go ahead and gather featers
    #     if songURI:
    # pull audio features for a songURI
    features = sp.audio_features(songURI)

#     spotFeatures['key'].append(features[0]['key'])
    spotFeatures['tempo'].append(features[0]['tempo'])
#     spotFeatures['duration_ms'].append(features[0]['duration_ms'])
    spotFeatures['time_signature'].append(features[0]['time_signature'])
    spotFeatures['energy'].append(features[0]['energy'])

    # append the difficulty label
    # difficultyLabel.append(dfSong[0][3].split(',')[1])

    # get the chord information
#     uniqChords = dfSong[0][2].split(',')[1:]
    uniqChords = dfSong['Chords'][iSong].split(',')[1:]
    nUniqueChords.append(len(uniqChords))

    tmpHard = []
    for kCh in range(0, nUniqueChords[iSong]):
        tmpHard.append(uniqChords[kCh] not in standardList)

    nHardChords.append(sum(tmpHard));


In [86]:
spotFeatures = pd.DataFrame.from_dict(spotFeatures)
# spotFeatures.head()
allFeatures = spotFeatures

allFeatures['n_unique_chords']    = nUniqueChords
allFeatures['n_difficult_chords'] = nHardChords

# allFeatures.drop(['key', 'duration_ms'], axis=1, inplace=True)
allFeatures

,tempo,time_signature,energy,n_unique_chords,n_difficult_chords
0,113.139,4,0.673,4,0
1,113.253,4,0.805,2,0
2,113.253,4,0.805,6,1
3,125.972,4,0.980,3,0
4,113.375,4,0.833,3,0
5,97.927,4,0.919,3,0
6,115.991,4,0.984,6,0
7,129.595,4,0.631,12,3
8,129.595,4,0.631,4,0
9,168.808,4,0.828,5,1


In [87]:
dfKnownSong = pd.DataFrame(allFeatures.iloc[0]).transpose()

In [88]:
import pickle
# load the model from disk
filename = 'dtree_model_0924.sav'
dtree = pickle.load(open(filename, 'rb'))

novel_song_label = dtree.predict(allFeatures)
novel_song_label

array([2, 1, 2, 0, 1, 2, 0, 2, 0, 0, 2, 2, 2, 0, 1, 2, 0, 0, 2, 0, 2, 1,
       2, 0, 0, 2, 1, 0, 0, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 1, 1, 1, 2, 2,
       0, 0, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2, 2])

In [151]:
clist   = dfSong['Chords'][0].split(',')
nchords = len(clist)

nSameChords     = []
propSameChords  = []
exactSame       = []
similar_nchords = []
sim_nchords_ix  = []
for iSong in range(1, len(dfSong)):
    similar_nchords.append(len(dfSong['Chords'][iSong].split(',')) <= nchords )

#    if similar_nchords[iSong-1]==True & novel_song_label[iSong-1] == 0:
#     if similar_nchords[iSong-1]==True & novel_song_label[0] == novel_song_label[iSong-1]:
    if similar_nchords[iSong-1]==True:
        if float(novel_song_label[0]) == float(novel_song_label[iSong-1]):
            sim_nchords_ix.append(iSong)
            nSameChords.append( len([ele for ele in clist if(ele in dfSong['Chords'][iSong].split(','))]) )
            propSameChords.append( len([ele for ele in clist if(ele in dfSong['Chords'][iSong].split(','))]) / len(dfSong['Chords'][iSong].split(','))  )
#     nSameChords.append( len([ele for ele in clist if(ele in dfSong['Chords'][iSong].split(','))]) )
#     exactSame.append(nSameChords[iSong-1]==len(clist))

In [152]:
import numpy as np

ixMostSame = np.argmax(propSameChords)
ixMostSame = sim_nchords_ix[ixMostSame]

dfSong.iloc[ixMostSame]

Song      edge-of-seventeen
Artist         stevie-nicks
Chords               Em,D,C
Name: 23, dtype: object

In [145]:
dfSong.iloc[0]

Song      island-in-the-sun
Artist               weezer
Chords          Am,Em,D,C,G
Name: 0, dtype: object

In [146]:
print( [novel_song_label[0] , novel_song_label[ixMostSame]])

float(novel_song_label[0]) == float(novel_song_label[ixMostSame])

[2, 0]


False

In [101]:
'island-in-the-sun' in list(dfSong['Song'])

True

In [102]:
pl_tracks

['Whole Lotta Love - 1990 Remaster',
 'All Along the Watchtower',
 'Back In Black',
 'Paint It, Black - Mono',
 'Rebel Rebel - 1999 Remaster',
 'Come Together - Remastered 2009',
 'Should I Stay or Should I Go - Remastered',
 "Baba O'Riley",
 'Sweet Emotion',
 'Another One Bites The Dust - Remastered 2011',
 'Light My Fire',
 'La Grange - 2005 Remaster',
 'American Girl',
 'Paradise City',
 'House Of The Rising Sun',
 'Like a Rolling Stone',
 'Sunshine Of Your Love',
 'Sweet Home Alabama',
 'Fortunate Son',
 'Another Brick in the Wall, Pt. 2',
 'The Chain',
 'Me and Bobby McGee',
 'Hotel California - 2013 Remaster',
 'The Joker',
 'Born To Be Wild',
 "For What It's Worth",
 'More Than a Feeling',
 'Sultans of Swing',
 'Somebody to Love',
 'Walk On the Wild Side',
 'Rock And Roll All Nite',
 'Tom Sawyer',
 'Under Pressure - Remastered',
 'When Doves Cry',
 'Every Breath You Take - Remastered 2003',
 "(Don't Fear) The Reaper",
 'A Horse with No Name',
 "Rocket Man (I Think It's Going To 

In [110]:
print(dfSong.iloc[ixMostSame]['Song'].replace('-', ' ').title())
print(dfSong.iloc[ixMostSame]['Artist'].replace('-', ' ').title())
print(dfSong.iloc[ixMostSame]['Chords'])

Hurricane
Bob Dylan
Am,Em,C,F,Dm,G


In [109]:
dfSong.iloc[ixMostSame]

Song           hurricane
Artist         bob-dylan
Chords    Am,Em,C,F,Dm,G
Name: 28, dtype: object

In [119]:
len(novel_song_label)

57